In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#read in all the names
words = open('names.txt').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
len(words)

32033

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
#build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?
X,Y = [],[]
for w in words:
    # print(w)
    context = [0]*block_size
    for ch in w+'.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context),'-->',itos[ix])
        context.pop(0)
        context.append(ix)
X = torch.tensor(X)
Y = torch.tensor(Y)

In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [7]:
C = torch.randn((27,2))

In [8]:
F.one_hot(torch.tensor(5), num_classes=27).dtype

torch.int64

In [9]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([0.1963, 0.7793])

In [10]:
C[5]

tensor([0.1963, 0.7793])

In [11]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [12]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [13]:
torch.cat(torch.unbind(emb,1), 1).shape

torch.Size([228146, 6])

In [17]:
# emb.view(32,6)==torch.cat(torch.unbind(emb,1), 1)

RuntimeError: shape '[32, 6]' is invalid for input of size 1368876

In [ ]:
h = torch.tanh(emb.view(-1,6)@W1 + b1)

In [15]:
h.shape

NameError: name 'h' is not defined

In [16]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)

In [23]:
logits = h@W2 + b2

In [24]:
logits.shape

torch.Size([32, 27])

In [25]:
counts = logits.exp()

In [26]:
probs = counts/counts.sum(1,keepdims=True)

In [27]:
probs.shape

torch.Size([32, 27])

In [28]:
probs[0].sum()

tensor(1.)

In [30]:
Y.shape

torch.Size([32])

In [33]:
#checking the current probabilities for the correct value
probs[torch.arange(32), Y]

tensor([2.5356e-14, 9.1420e-07, 9.1420e-07, 4.9083e-16, 4.5948e-09, 8.4550e-02,
        4.9714e-07, 1.9030e-06, 2.3874e-14, 1.9030e-06, 1.6222e-12, 1.6793e-10,
        1.5940e-11, 1.7764e-07, 1.5940e-11, 2.3402e-04, 1.1117e-15, 2.4558e-09,
        2.6307e-16, 2.7049e-13, 2.6748e-14, 3.9298e-10, 3.9298e-10, 2.6307e-16,
        6.7742e-11, 5.0734e-01, 8.4550e-02, 7.3692e-03, 1.0129e-07, 1.9030e-06,
        1.6222e-12, 1.6793e-10])

In [34]:
loss = -probs[torch.arange(32), Y].log().mean()

In [35]:
loss

tensor(20.3581)

##### Cleaner version

In [22]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [23]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [24]:
sum( p.nelement() for p in parameters) #no. of parameters in total

3481

In [25]:
for p in parameters:
    p.requires_grad = True

In [26]:
for _ in range(10000):
    #forward pass
    #minibatch construct
    ix = torch.randint(0, X.shape[0], (32,)) # the training is much faster now!
    #went from taking a couple of seconds to being almost instantaneous!
    #because we are estimating only from 32 examples, the quality of the gradient is lower, so the direction of the gradient is not as reliable
    #but we assume that the gradient is good enough that it is useful
    #much better to have an approximate gradient and take more steps
    #than it is to evaluate the exact gradient and take fewer steps.
    
    emb = C[X[ix]] #(32,3,2)
    h = torch.tanh(emb.view(-1,6)@W1 + b1)
    logits = h@W2 + b2
    # counts = logits.exp()
    # probs = counts/counts.sum(1,keepdims=True)
    # loss = -probs[torch.arange(32), Y].log().mean()
    #much more efficient (used for classification)
    #no new tensors made
    #backward pass is made more efficient
    #the results are well behaved, =>
        #if one of the logits is 100, -> counts would have value inf for that logit
        #impossible to calculate prob. and also the loss
    loss = F.cross_entropy(logits,Y[ix]) 
    # print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    lr = 0.01
    for p in parameters:
        p.data += -lr * p.grad
print(loss.item())


2.751224994659424


In [27]:
emb = C[X]
h = torch.tanh(emb.view(-1,6)@W1 + b1)
logits = h@W2 + b2
loss = F.cross_entropy(logits,Y)
loss

tensor(2.7815, grad_fn=<NllLossBackward0>)

In [28]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0])